In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import torchvision.utils as vutils
from torch.utils.data import WeightedRandomSampler, DataLoader, Dataset
import time
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd

import seaborn as sns

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('running on:', device)

running on: cuda:0


In [3]:
IMG_SIZE = 48

CLASSES = ['neutral', 'happy', 'surprised', 'sad', 'angry'] #, 'disgusted', 'afraid'
NUM_CLASSES = len(CLASSES)

## Pre-processing

In [4]:
df_fer = pd.read_csv('../data/fer2013.csv').iloc[:, 1:]
df_ferplus = pd.read_csv('../data/ferplus2013.csv').iloc[:, [2, 3, 4, 5, 6]]
df = df_fer.join(df_ferplus)
# df.head(15)

df['emotion'] = df.iloc[:, 2:].idxmax(axis=1).tolist()
df = df.replace(dict(zip(pd.Series(CLASSES),pd.Series(CLASSES).index)))
# df.head(15)

print('number of images:', df.shape[0])
df = df[(df.iloc[:, 2:-1].max(axis=1) > 5)]
print('after removing images with unknown/unsure classification:', df.shape[0])
# df.head(15)

df = df.iloc[:, [0, 1, -1]]
# df.head(15)

number of images: 35887
after removing images with unknown/unsure classification: 26811


In [5]:
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.545), (0.245)),
])

images_np = np.array(df['pixels'])
label_np = np.array(df['emotion'])

images = []
labels = []

for i in range(df.shape[0]):
    b = bytes(int(p) for p in images_np[i].split())
    img = Image.frombuffer('L', (IMG_SIZE, IMG_SIZE), b)
    images.append(transform(img))
    labels.append(label_np[i])

In [6]:
label_counts = [0]*NUM_CLASSES

for i in range(len(labels)):
    label_counts[labels[i]] += 1

print('\033[4m{: <15} {: >10}\033[0m'.format('class', '#images'))
for i in range(NUM_CLASSES):
    print('{: <15} {: >10}'.format(CLASSES[i], label_counts[i]))

class              #images
neutral               9494
happy                 8802
surprised             3461
sad                   2958
angry                 2096


In [7]:
train_split_pct = 0.7
train_len = int(len(labels) * train_split_pct)

train_images = images[:train_len]
train_labels = labels[:train_len]
test_images = images[train_len:]
test_labels = labels[train_len:]

print(f'train length: {len(train_labels)} ({len(train_labels)*100/len(labels):.1f}%)')
print(f'test length: {len(test_labels)} ({len(test_labels)*100/len(labels):.1f}%)')

train length: 18767 (70.0%)
test length: 8044 (30.0%)


In [8]:
BATCH_SIZE = 128

class CustomDataset(Dataset):
    
    def __init__(self, images, labels):      
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]


train_dataset = CustomDataset(train_images, train_labels)
test_dataset = CustomDataset(test_images, test_labels)


def get_loader(dataset):
    print('get_loader():')

    label_counts = [0]*NUM_CLASSES
    for i in range(len(dataset.labels)):
        label_counts[dataset.labels[i]] += 1

    class_weights = pow(np.array(label_counts) / sum(label_counts), -1)
    print('  class weights:', class_weights.round(3), '\n')

    sample_weights = [0]*len(dataset)
    for idx, (image, label) in enumerate(dataset):
        sample_weights[idx] = class_weights[label]

    sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
    return DataLoader(dataset, BATCH_SIZE, sampler=sampler, shuffle=False)


train_loader = get_loader(train_dataset)
test_loader = get_loader(test_dataset)

get_loader():
  class weights: [ 2.823  3.008  7.849  9.044 13.263] 

get_loader():
  class weights: [ 2.825  3.139  7.518  9.11  11.812] 



-----
# AlexNet Model

In [9]:
# Batch size during training, remember to change the global one as well
batch_size = 128

# Number of training epochs
num_epochs = 30

# Learning rate for optimizers
lr = 0.001

In [10]:
# Function for displaying predictions for a few images, in this case 6.
#print(test_set[0])

class_names = CLASSES
print(class_names)

def visualize_expression_model(model, num_images=4):
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(test_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                plt.imshow(inputs.cpu().data[j].permute(1, 2, 0), cmap="gray")

                if images_so_far == num_images:
                    return

['neutral', 'happy', 'surprised', 'sad', 'angry']


In [11]:
def train_expression_model(model, data, criterion, optimizer):
    since = time.time()
    best_acc = 0.0
    model.train()   # Set model to train mode
    running_loss = 0.0
    running_corrects = 0
    
    for inputs, labels in data:
        inputs = inputs.to(device)
        labels = labels.to(device)
    
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        # zero the parameter gradients, and use backpropagatation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels.data)
        
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() / len(train_dataset)

    print('train Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

In [12]:
def test_expression_model(model, data, criterion):
    since = time.time()
    best_acc = 0.0
    model.eval()   # Set model to evaluate mode
    running_loss = 0.0
    running_corrects = 0
    
    for inputs, labels in data:
        inputs = inputs.to(device)
        labels = labels.to(device)
    
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels.data)
        
    epoch_loss = running_loss / len(test_dataset)
    epoch_acc = running_corrects.double() / len(test_dataset)

    print('test Loss: {:.4f} Acc: {:.4f}'.format(
        epoch_loss, epoch_acc))

In [13]:
# This is a recreation of the AlexNet model

class Facial_Expression_Network(nn.Module):
    def __init__(self):
        super(Facial_Expression_Network, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, IMG_SIZE, kernel_size=(3,3), stride=(1, 1), padding=(2, 2)), 
            nn.ReLU(inplace=True), # Rectified Linear Unit activation function
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False), # pooling layer for reducing dimensions
            nn.Conv2d(IMG_SIZE, 192, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2)),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
        )

        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(6, 6)) # Applies a 2D adaptive average pooling over an input composed of several input planes.


        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5, inplace=False), # Dropout layer for setting 50% of the activations to 0, fording the network to not rely on any 1 node
            nn.Linear(in_features=9216, out_features=4096, bias=True), # Linear layer
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=4096, out_features=5, bias=True),
        ) # ---- Change out_features if change in amount of classes ----


        self.softmax = nn.Softmax(dim=1) # last activation function for the network, normalizing the output
        self.softmax_result = 0 

    def forward(self, x, verbose=False):
        x = self.features(x)
        x = self.avgpool(x)

        # The data needs to be flattened after the AdaptiveAvgPool2d as its output is H x W
        # This is because the classifier's first layer is a Linear layer
        x = torch.flatten(x, 1)

        x = self.classifier(x) 
        self.softmax_result = self.softmax(x)
        return x

In [16]:
expression_model = Facial_Expression_Network().to(device)

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(expression_model.parameters(), lr=lr, momentum = 0.9) # So far, only 31.1% acc
#optimizer = torch.optim.Adam(expression_model.parameters(), lr=lr)

# Decay LR by a factor of 0.1 every 1 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

In [17]:
for epoch in range(1, (num_epochs+1)):
    print('Epoch {}/{}'.format(epoch, num_epochs))
    print()
    train_expression_model(expression_model, train_loader, loss_fn, optimizer)
    test_expression_model(expression_model, test_loader, loss_fn)
    scheduler.step

Epoch 1/30

train Loss: 0.0126 Acc: 0.1969
test Loss: 0.0126 Acc: 0.1965
Epoch 2/30

train Loss: 0.0126 Acc: 0.2034
test Loss: 0.0126 Acc: 0.2030
Epoch 3/30

train Loss: 0.0126 Acc: 0.2066
test Loss: 0.0126 Acc: 0.2075
Epoch 4/30

train Loss: 0.0126 Acc: 0.2101
test Loss: 0.0126 Acc: 0.2015
Epoch 5/30

train Loss: 0.0126 Acc: 0.2082
test Loss: 0.0126 Acc: 0.1988
Epoch 6/30



In [ ]:
visualize_expression_model(expression_model)

##### End of AlexNet Model
-----

In [ ]:
samples_train = [0]*NUM_CLASSES
samples_test = [0]*NUM_CLASSES

for idx, (images, labels) in enumerate(train_loader):
    for i in range(len(images)):
        samples_train[labels[i].item()] += 1

for idx, (images, labels) in enumerate(test_loader):
    for i in range(len(images)):
        samples_test[labels[i].item()] += 1


print('\033[4m{: <15} {: >15} {: >20}\033[0m'.format('class', '#samples (train)', '#samples (test)'))
for i in range(NUM_CLASSES):
    print('{: <15} {: >15} {: >20}'.format(CLASSES[i], samples_train[i], samples_test[i]))

In [ ]:
def imshow(dataset, title='', num_images=(3,6), rand=True):

    plt.figure(figsize=(num_images[1]*1.5, num_images[0]*2)); 
    plt.suptitle(title, fontsize=16)

    for i in range(num_images[0]):
        for j in range(num_images[1]):
            
            c = num_images[1]*i+(j+1)

            if not rand:
                idx = c
            else:
                idx = random.randint(0, len(dataset)-1)

            plt.subplot(num_images[0], num_images[1], c); plt.axis('off'); plt.title(CLASSES[dataset[idx][1]])
            plt.imshow(dataset[idx][0].permute(1,2,0), cmap='gray')

    plt.tight_layout(); plt.show()


imshow(train_dataset, 'Example images from the train dataset')